# Song Recommender System using Last.fm dataset

#### Task
* Build a song recommender system for female users in the United Kingdom.

#### Dataset
* [Last.fm](http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/index.html)

#### Algorithm
* Memory-based Collaborative Filtering (CF) Recommender System

There are two common approaches for collaborative filtering (CF) recommender systems, one is memory-based and one is model-based.  

In memory-based CF, we first compute item-user matrix in memory, calculate user cosine similarity (for user-based) or item cosine similarity (for item-based), and make prediction based on similarity measures (no learning is needed in this approach). The problems of memory-based CF are sparsity and scalability. To boost computational efficiency, we can leverage on sparse matrix package in scipy. This notebook demonstrates memory-based CF for song recommender system using last.fm dataset.  

## Memory-based CF from scratch (item-based / user-based)

* Step 1: Compute user-item matrix
* Step 2: Compute item cosine similarity / user cosine similarity
* Step 3: Make item-based / user-based predictions

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import pairwise_distances
from scipy.sparse import csr_matrix
import time

### Load and prepare data

In [2]:
# Load data
user_data_filepath = 'data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
user_profiles_filepath = 'data/lastfm-dataset-360K/usersha1-profile.tsv'

df_data = pd.read_table(user_data_filepath, header = None, 
                        names = ['user', 'musicbrainz-artist-id', 'artist', 'plays'],
                        usecols = ['user', 'artist', 'plays'])

df_user = pd.read_table(user_profiles_filepath,
                          header = None,
                          names = ['user', 'gender', 'age', 'country', 'signup'],
                          usecols = ['user', 'gender','country'])

len(df_data) #17535655
len(df_user) #359347

359347

We only use part of the dataset. First, limit the data set to female users in United Kingdom, then prepare the data into dataframes. 

In [3]:
# Limit the data set to female users in United Kingdom only
df_user_UK = df_user[df_user['country']=='United Kingdom'].drop('country', axis=1)
df_user_UK_female = df_user_UK[df_user_UK['gender']=='f'].drop('gender', axis=1)
len(df_user_UK_female) #5851

# Merge the two dataframes
df = df_data.merge(df_user_UK_female, left_on='user', right_on ='user', how='inner')
df = df.groupby(['user','artist'], as_index=False).sum()
len(df) # 288780

288780

For illustrative purpose, I only use top 1% artists (this also help remove noise).

In [4]:
# Find total number of plays for each artist
df_artist = df_data.groupby(['artist'])['plays'].sum().reset_index().rename(columns = {'plays': 'total_plays'})
df_artist.describe() 

# Find total number of plays of the 99th percentile artist
df_artist['total_plays'].quantile(0.99)  #198482.5899999995

# Set threshold = 200000 clicks, limit the data set to artists with more than 200000 clicks
df_top_artist = df_artist[df_artist['total_plays']>200000].sort('total_plays', ascending=False)
print("Top 10 artists: \n", df_top_artist[0:9])

top_artist = list(df_top_artist['artist'])
df = df[df['artist'].isin(top_artist)]
df.head()
len(df) #202917

/Users/sukilau/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Top 10 artists: 
                        artist  total_plays
252495            the beatles     30466827
217884              radiohead     27426234
73856                coldplay     16686772
212497             pink floyd     15943557
186241              metallica     15481852
194163                   muse     15451683
199674        nine inch nails     14075619
220129  red hot chili peppers     13547741
171169            linkin park     12836638


202917

### Compute user-item matrix

* user-item matrix dim = (num of users, num of items)

In [124]:
# Create item-user matrix, where each row is the artist i (item) and each column is the user j,
## and the entry is the number of total plays clicked by user j to artist i.
matrix = df.pivot(index ='artist', columns='user', values='plays').fillna(0)
matrix_sparse = csr_matrix(matrix)

In [ ]:
# checking
matrix.shape #(30593, 5841)   (2840, 5828)-top1%
matrix.index.get_loc('radiohead') #1976
matrix.index[1976]  #'radiohead'
matrix.iloc[1976]
matrix.loc["radiohead"] #same as above

### Compute cosine similarity between users and items 

* item_similarity matrix dim = (num of items, num of items)
* user_similarity matrix dim = (num of users, num of users)

In [6]:
item_similarity = pairwise_distances(matrix_sparse, metric='cosine')
user_similarity = pairwise_distances(matrix_sparse.T, metric='cosine')

In [22]:
# checking
item_similarity.shape #(2840, 2840)
user_similarity.shape #(5828, 5828)

(1, 2840)

### Make prediction

* item_prediction matrix dim = (num of users, num of items)
* user_prediction matrix dim = (num of users, num of items)

In [153]:
# Make prediction
def predict(matrix, similarity, type='user'):
    if type == 'user':
        mean_user_rating = matrix.mean(axis=1)
        ratings_diff = (matrix - mean_user_rating)
        pred = mean_user_rating + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = matrix.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

item_prediction = predict(matrix_sparse.T, item_similarity, type='item')
user_prediction = predict(matrix_sparse.T, user_similarity, type='user')

## Memory-based CF using KNNBasic in Surprise

We can also use [Surprise](http://surprise.readthedocs.io/en/stable/getting_started.html) (Recommender System library) to implement KNN-inspired algorithms for memory-based CF.

In [205]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, KNNBasic, evaluate

data = Dataset.load_from_df(df[['user', 'artist', 'plays']], Reader(rating_scale=(1, df['plays'].max())))

# First, train the algortihm to compute the similarities between items
# training is very slow..
trainset = data.build_full_trainset()

# compute cosine similarities between items
sim_options = {'name': 'cosine','user_based': False}
knn = KNNBasic(k=5, sim_options=sim_options)
knn.train(trainset)

# predict a certain item
userid = '000cdac4253f2dcecbd3dff8c5d7cf8cf0426c7a'
itemid = 'john mayer'
print(knn.predict(userid, itemid))

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 000cdac4253f2dcecbd3dff8c5d7cf8cf0426c7a item: john mayer r_ui = None   est = 128.03   {'actual_k': 5, 'was_impossible': False}


In [201]:
# actual rating 
print("Actual no. of plays of john mayer by user 0 =", matrix.loc["john mayer"][0])

Actual no. of plays of john mayer by user 0 = 197.0
